In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


In [3]:
"""404개의 훈련샘플, 102개의 테스트샘플"""
train_data.shape
test_data.shape

(102, 13)

In [4]:
"""
입력데이터에 있는 각 특성은 스케일이 다르다.
어떤값은 0~1사이 비율, 어떤값은 1~12사이의 값, 어떤값은 1~100까지의 값
상이한 스케일 값을 신경망에 주입하면 문제가 되니 조정해줘야함
입력데이터에 있는 각 특성에 대해서, 즉 입력데이터 행렬의 열에 대해서 (axis=0) 특성의 평균을 빼고 표준편차로 나눈다.
테스트데이터를 정규화할때에도 훈련데이터의 mean, std값으로 계산한다.
---------머신러닝작업과정에서 절대로 테스트데이터에서 계산한 어떤값도 사용하면 안된다!
"""
mean = train_data.mean(axis=0)
train_data -=mean
std = train_data.std(axis=0)
train_data /=std

test_data -=mean
test_data/=std

len(train_data)

404

In [5]:
"""
훈련데이터의 갯수가 적을수록 과대적합이 더 쉽게 일어나므로 
데이터가 적을때에는 작은 모델을 사용하는것이 과대적합을 피하는 방법.
마지막층은 활성화 함수가 없다(선형층), 전형적인 스칼라회귀(하나의연속적인 값을 예측)구성
[마지막층에 sigmoid활성화 함수를 쓰면 0~1사이의 값이 나온다]
회귀문제에서 널리 사용되는 mse손실함수는 평균제곱오차의 약어로 예측과 타깃 사이 거리의 제곱이다.
훈련도중 모니터링을 위해 평균절대오차(MAE)를 측정. 예측과 타깃 사이 거리의 절댓값이다.
[예를들어 이 예제에서 MAE가 0.5이면 예측이 평균적으로 500달러 정도 차이난다는 뜻.]
"""
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],) ))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [9]:
"""
k겹 교차검증
검증세트와 훈련세트로 어떤 데이터 포인트가 선택되었는지에 따라 검증점수가 크게 달라지기때문에 
데이터를 K개(일반적으로 k = 4 또는 5) 분할로나누고 K개의 모델을 만들어 k-1개의 분할에서 훈련, 나머지 분할에서 평가.
검증점수는 k개의 검증점수의 평균.
"""
import numpy as np

k=4
num_val_samples = len(train_data)//k
num_epochs = 100
all_scores=[]
for i in range(k):
    print('처리중인 폴드 #',i)
    val_data = train_data[i * num_val_samples: (i+1)* num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
     [train_data[:i * num_val_samples], 
      train_data[(i+1)*num_val_samples:]],
    axis=0)
    
    partial_train_targets = np.concatenate(
    [train_targets[:i*num_val_samples],
    train_targets[(i+1)*num_val_samples:]],
    axis=0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
             epochs = num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


In [11]:
np.mean(all_scores)

2.4340372075243755